In [ ]:
% pip install optuna

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import optuna
import numpy as np
from functools import partial
import pandas as pd

optuna.logging.set_verbosity(optuna.logging.WARNING) # removes trial text

# Importing excel sheets as dfs, should both have samples in the same order
predictors_df = pd.read_excel('../Predictors_Cleaned.xlsx', 0)
    # Rows are samples, columns are predictors
outcomes_df = pd.read_excel('../Outcomes_Cleaned.xlsx', 0)
    # Rows are samples, columns are mechanisms

# Dropping unnecessary metadata
X = predictors_df.drop('SAMPLE NAME', axis=1)
Y = outcomes_df.drop('MECHANISM', axis=1) # Full DF must iterate through

columns = list(Y)
# x_train, x_test, y_train, y_test = train_test_split(X, Y['Metronidazole'], random_state=0, shuffle=True, test_size=0.4)

In [ ]:
# Graphing function
def graph_r2(r2_list, model_type):
    fig, ax = plt.subplots(figsize=(20,10))
    
    # Sorting r2 values for visualization
    sorted_idx = np.argsort(r2_list)
    r2_array = np.array(r2_list)
    
    # Sorting colors for visualization
    bar_labels = ['Drugs', 'Metals', '_Metals', '_Drugs', '_Drugs', '_Metals', '_Metals', 'Biocides', 'Multi-drug Resistance', '_Biocides', '_Biocides', '_Drugs', '_Drugs', '_Biocides', '_Biocides', '_Drugs', '_Drugs', '_Metals', 'Multi-biocide Resistance', '_Drugs', '_Drugs', '_Drugs', '_Drugs', '_Drugs', 'Drug & Biocide Resistance', 'Multimetal Resistance', '_Metals', '_Drugs', '_Drugs', 'Drug & Biocide & Metal Resistance', '_Drugs', 'Biocide & Metal Resistance', '_Metals', '_Biocides', '_Drugs', '_Biocides', '_Drugs', '_Metals', '_Metals', '_Biocides']
    formatted_genes = ['Lipopeptides', 'Arsenic Resistance', 'Copper Resistance', 'Rifampin',
           'Trimethoprim', 'Tellurim Resistance', 'Zinc Resistance',
           'Peroxide Resistance', 'Multi-drug Resistance', 'Acid Resistance',
           'Phenolic Compound Resistance', 'Mupirocin', 'Nucleosides',
           'Aldehyde Resistance', 'Paraquat Resistance', 'Betalactams',
           'Tetracyclines', 'Nickel Resistance', 'Multibiocide Resistance',
           'Aminoglycoside', 'MLS', 'Fluoroquinolones', 'Fosfomycin',
           'Phenicol Resistance', 'Drug And Biocide Resistance',
           'Multimetal Resistance', 'Mercury Resistance', 'Glycopeptides',
           'Sulfonamides', 'Drug And Biocide And Metal Resistance',
           'Cationic Microbial Peptides', 'Biocide And Metal Resistance',
           'Iron Resistance', 'Bioguanide Resistance', 'Metronidazole',
           'Acetate Resistance', 'Bicitracin', 'Sodium Resistance',
           'Chromium Resistance', 'QACS']
    formatted_genes = np.array(formatted_genes)
    
    drug_color = '#ef476f'
    metal_color = '#99ccff'
    biocide_color = '#83d483'
    multi_drug_color = '#ffd166'
    multi_metal_color = '#f78c6b'
    multi_biocide_color = '#118ab2'
    drug_biocide_color = '#c77dff'
    drug_biocide_metal_color = "#f695ff"
    metal_biocide_color = "#2C2C2C"
    
    bar_colors = [f'{drug_color}', f'{metal_color}', f'{metal_color}', f'{drug_color}', f'{drug_color}', f'{metal_color}', f'{metal_color}', f'{biocide_color}', f'{multi_drug_color}', f'{biocide_color}', f'{biocide_color}', f'{drug_color}', f'{drug_color}', f'{biocide_color}', f'{biocide_color}', f'{drug_color}', f'{drug_color}', f'{metal_color}', f'{multi_biocide_color}', f'{drug_color}', f'{drug_color}', f'{drug_color}', f'{drug_color}', f'{drug_color}', f'{drug_biocide_color}', f'{multi_metal_color}', f'{metal_color}', f'{drug_color}', f'{drug_color}', f'{drug_biocide_metal_color}', f'{drug_color}', f'{metal_biocide_color}', f'{metal_color}', f'{biocide_color}', f'{drug_color}', f'{biocide_color}', f'{drug_color}', f'{metal_color}', f'{metal_color}', f'{biocide_color}']
    # [f'tab:{drug_color}', f'tab:{metal_color}', f'tab:{metal_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{metal_color}', f'tab:{metal_color}', f'tab:{biocide_color}', f'tab:{multi_drug_color}', f'tab:{biocide_color}', f'tab:{biocide_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{biocide_color}', f'tab:{biocide_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{metal_color}', f'tab:{multi_biocide_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{drug_biocide_color}', f'tab:{multi_metal_color}', f'tab:{metal_color}', f'tab:{drug_color}', f'tab:{drug_color}', f'tab:{drug_biocide_metal_color}', f'tab:{drug_color}', f'tab:{metal_biocide_color}', f'tab:{metal_color}', f'tab:{biocide_color}', f'tab:{drug_color}', f'tab:{biocide_color}', f'tab:{drug_color}', f'tab:{metal_color}', f'tab:{metal_color}', f'tab:{biocide_color}']
    
    # Plotting graph
    ax.barh(formatted_genes[sorted_idx], r2_array[sorted_idx], align='center', label=np.array(bar_labels)[sorted_idx], color=np.array(bar_colors)[sorted_idx])
    # ax.barh(np.arange(len(columns)), r2_list, align='center') # old code
    ax.set_yticks(np.arange(len(columns)), labels=formatted_genes[sorted_idx])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('R-Squared')
    ax.set_ylabel('Gene Class')
    plt.xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_title(f'{model_type} Model Comparison by R-Squared Values')
    ax.legend(title='Gene Type')
    
    plt.savefig(f'Graphs/{model_type}_r2_graph.png')
    # plt.show()

In [ ]:
# XGBoost Objective function
def xgb_objective(trial, i):
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 0
    }
    test_size = trial.suggest_float('test_size', 0.2, 0.8)

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, Y[i], test_size=test_size, random_state=0)

    model = XGBRegressor(**param)  
    model.fit(x_train, y_train)
    
    predict = model.predict(x_test)
    r_2 = abs(r2_score(y_test, predict))

    if r_2 > 1:
        r_2 = 0 # invalid run
    
    return r_2

In [ ]:
# XGBoost Iterations # https://www.kaggle.com/code/alisultanov/regression-xgboost-optuna
xgb_r2_list = []

for i in columns: # iterate through gene classes
    print(i)
    # Create study
    study = optuna.create_study(direction="maximize") # finding max r_2
    objective = partial(xgb_objective, i = i) # using partial params
    study.optimize(objective, n_trials=50, show_progress_bar=True)

    # Print results
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value)) # r2
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save r2 for visualization
    xgb_r2_list.append(trial.value)

# Graph results
graph_r2(xgb_r2_list, "XGBoost")

In [ ]:
# RF Objective Function # https://www.kaggle.com/code/mustafagerme/optimization-of-random-forest-model-using-optuna
def rf_objective(trial, i):
    # Hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    test_size = trial.suggest_float('test_size', 0.2, 0.8)

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, Y[i], test_size=test_size, random_state=0)

    # Model random forest
    model = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=0,
    )
    model.fit(x_train, y_train)

    # Calculate r2
    predict = model.predict(x_test)
    r_2 = abs(r2_score(y_test, predict))

    if r_2 > 1:
        r_2 = 0 # invalid run

    return r_2

In [ ]:
# Random Forest Iterations
rf_r2_list = []

for i in columns: # iterate through gene classes
    print(i)
    # Create study
    study = optuna.create_study(direction="maximize") # finding max r_2
    objective = partial(rf_objective, i = i) # using partial params
    study.optimize(objective, n_trials=50, show_progress_bar=True)

    # Print results
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value)) # r2
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save r2 for visualization
    rf_r2_list.append(trial.value)

# Graph results
graph_r2(rf_r2_list, "RandomForest")

In [ ]:
# Lasso Objective Function
def lasso_objective(trial, i):
    # Hyperparameters
    alpha = trial.suggest_float("alpha", 0.0001, 0.01)
    max_iter = trial.suggest_int("max_iter", 100000, 10000000)
    test_size = trial.suggest_float('test_size', 0.2, 0.8)

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, Y[i], test_size=test_size, random_state=0)
    
    # Model Lasso
    lasso = linear_model.Lasso(alpha=alpha, max_iter=max_iter, random_state=0)
    lasso = lasso.fit(x_train, y_train)

    # Calculating r2
    predict = lasso.predict(x_test)
    r_2 = abs(r2_score(y_test, predict))

    if r_2 > 1:
        r_2 = 0 # invalid run
        
    return r_2

In [ ]:
# Lasso Iterations
lasso_r2_list = []

for i in columns: # iterate through gene classes
    print(i)
    # Create study
    study = optuna.create_study(direction="maximize") # finding max r_2
    objective = partial(lasso_objective, i = i) # using partial params
    study.optimize(objective, n_trials=50, show_progress_bar=True)

    # Print results
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value)) # r2
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save r2 for visualization
    lasso_r2_list.append(trial.value)

# Graph results
graph_r2(lasso_r2_list, "Lasso")

In [ ]:
# PCA Objective Function
def pca_objective(trial, i):
    # Hyperparameters
    alpha = trial.suggest_float("alpha", 0.0001, 0.01)
    max_iter = trial.suggest_int("max_iter", 100000, 10000000)
    test_size = trial.suggest_float('test_size', 0.2, 0.8)

    # Split the data
    x_train, x_test, y_train, y_test = train_test_split(X, Y[i], test_size=test_size, random_state=0)
    
    # Model Lasso
    lasso = linear_model.Lasso(alpha=alpha, max_iter=max_iter, random_state=0)
    lasso = lasso.fit(x_train, y_train)

    # Calculating r2
    predict = lasso.predict(x_test)
    r_2 = abs(r2_score(y_test, predict))

    if r_2 > 1:
        r_2 = 0 # invalid run
        
    return r_2